In [4]:
import numpy as np
import os
from tqdm import tqdm
import requests
import matplotlib.pyplot as plt
import json

In [202]:
'''
Funcão utilizada para computar o valor de ajuste de cada solução da atual
população. Retorna a lista de [chomossomo,valor_de_ajuste] e prob_fits que
contém a probabilidade de cada cromossomo para a fase de seleção
'''

def fitness_function(population):
    evaluated = []
    for chromosome in population:
        #resp = requests.get('https://aydanomachado.com/mlclass/02_Optimization.php?phi1={}&theta1={}&phi2={}&theta2={}&phi3={}&theta3={}&dev_key=Andre%20Santos'.format(chromosome[0],chromosome[1],chromosome[2],chromosome[3],chromosome[4],chromosome[5]))
        resp = requests.get('http://localhost:8080/antenna/simulate?phi1={}&theta1={}&phi2={}&theta2={}&phi3={}&theta3={}'.format(chromosome[0],chromosome[1],chromosome[2],chromosome[3],chromosome[4],chromosome[5]))
        resp = resp.text
        #resp = resp.replace("'", "\"")
        #resp = json.loads(resp)
        #resp = resp['gain']
        resp = float(resp.split('\n')[0])
        evaluated.append([chromosome,resp])
    
    evaluated.sort(key=lambda x:x[1], reverse=True)
    prob_rank = list(np.arange(len(evaluated)))
    return evaluated,prob_rank

In [185]:
'''
Função que faz o cruzamento de dois cromossomos dados, retorna dois cromossomos filhos
'''
def crossover(father1,father2):
    #child1 = [np.hstack((father1[0][:3],father2[0][3:])),0.0]
    #child2 = [np.hstack((father2[0][:3],father1[0][3:])),0.0]
    #print('f1={}, f2={}'.format(father1,father2))
    child1 = [father1[:2]+father2[2:4]+father1[4:],0.0]
    child2 = [father2[:2]+father1[2:4]+father2[4:],0.0]

    return child1,child2

In [195]:
def roulettewheel(current_gen_ord,probs):
    
    choice = np.random.uniform(0,probs[-1])
    count_sum = 0
    i = 0
    
    for prob in probs:
        count_sum += prob
        if count_sum >= choice:
            return current_gen_ord[i]
        i+=1
        

In [196]:
'''
Função que aplicaa mutação a um cromossomo dado, a probabilidade de mutação para um 
determinado cromossomo é de 5%
'''
def mutation(new_generation):

    prob_mutation = 5
    muted = []
    for chrom in new_generation:
        if np.random.randint(0,100) <= prob_mutation:
            position = np.random.randint(0,5)
            chrom[position] = np.random.randint(0,359)
            muted.append(chrom)
        else:
            muted.append(chrom)
    return muted

In [206]:
def selection(ordened_population,probs,n_parents):

    next_generation = []
    n_crossover = 0
    
    father1 = None
    father2 = None
    
    while True:
        if n_crossover == n_parents:
            break
        father1 = roulettewheel(ordened_population[::-1],probs[::-1])
        #while True:
        father2 = roulettewheel(ordened_population[::-1],probs[::-1])
        #if not (father2[0] == father1[0]):
            #break
        child1,child2 = crossover(father1[0],father2[0])
        
        #if not ((child1[0] in next_generation) and (child2[0] in next_generation)):
        next_generation.append(child1[0])
        next_generation.append(child2[0])
        n_crossover+=1
    return next_generation

In [177]:
def genetic_algorithm(population_init,n_gen,n_crossover):
    
    current_gen = population_init
    best_config = None
    best_value_fits = -float('inf')
    
    for gen in range(n_gen):
        current_gen,probs = fitness_function(current_gen)
        if best_value_fits < current_gen[0][1]:
            best_config = current_gen[0][0]
            best_value_fits = current_gen[0][1]
        selected = selection(current_gen,probs,n_crossover)
        muted = mutation(selected)
        next_gen = muted
        current_gen = next_gen
        
        #print('best = {}'.format(best_value))
    return best_config,best_value_fits 

In [207]:
size_pop = 10

init_pop = np.random.randint(0,359,(size_pop,6)).tolist()
n_gen = 50

best_config, best_value = genetic_algorithm(init_pop, n_gen,size_pop // 2 + 1)
print('Melhor configuração = {}, melhor valor = {}'.format(best_config, best_value))

Melhor configuração = [56, 235, 288, 63, 160, 70], melhor valor = 3.8874067871213285


In [200]:
list(np.arange(10))

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]